In [1]:
import os,sys
currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [2]:
import pennylane as qml
import torch
import numpy as np

In [3]:
import time

In [4]:
from utils.data_handling import simple_np_ds

In [5]:
dset=simple_np_ds("moons.npz",val_split=0.1,test_split=0.1)

In [6]:
len(dset)

1000

In [7]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [8]:
train_loader=DataLoader(dset,batch_size=4,shuffle=False,sampler=SubsetRandomSampler(dset.train_indices),num_workers=4)
val_loader=DataLoader(dset,batch_size=1,shuffle=False,sampler=SubsetRandomSampler(dset.val_indices),num_workers=4)
test_loader=DataLoader(dset,batch_size=1,shuffle=False,sampler=SubsetRandomSampler(dset.test_indices),num_workers=4)

In [9]:

def loop_over_set(loader,loop_limit=3):

    # Let's measure time that takes in each loop
    trecord = np.zeros([loop_limit],dtype=np.float32)
    t = time.time()
    for iteration, batch in enumerate(loader):

        data,labels = batch

        # Print out some content info
        print('Iteration',iteration,'... time:',time.time()-t,'[s]')
        print('data: {} labels: {}'.format(data,labels))

        trecord[iteration] = time.time() - t
        t = time.time()

        # break when reaching the loop limit
        if (iteration+1) == loop_limit:
            break
    return trecord

In [10]:
loop_over_set(train_loader,10)

Iteration 0 ... time: 0.059876441955566406 [s]
data: tensor([[-0.6231,  0.1443],
        [-0.9353, -0.0446],
        [ 1.5164, -0.3222],
        [ 0.0951, -0.2689]]) labels: tensor([0, 0, 1, 1])
Iteration 1 ... time: 0.0033025741577148438 [s]
data: tensor([[ 0.2689, -0.1982],
        [ 0.0521,  1.0190],
        [-0.4111,  1.0237],
        [ 1.1878, -0.4956]]) labels: tensor([1, 0, 0, 1])
Iteration 2 ... time: 0.002005338668823242 [s]
data: tensor([[-0.3922,  1.0957],
        [ 0.8930, -0.5334],
        [ 1.6755, -0.3622],
        [ 0.5163,  0.6125]]) labels: tensor([0, 1, 1, 0])
Iteration 3 ... time: 0.00013971328735351562 [s]
data: tensor([[-0.8767,  0.6248],
        [-1.0204,  0.0132],
        [ 0.3113,  0.6965],
        [ 1.3908, -0.4691]]) labels: tensor([0, 0, 1, 1])
Iteration 4 ... time: 0.00016164779663085938 [s]
data: tensor([[-1.1643,  0.3249],
        [-0.2860,  0.9257],
        [-0.0817,  0.2431],
        [ 0.9261, -0.4002]]) labels: tensor([0, 0, 1, 1])
Iteration 5 ... time

array([0.06490684, 0.00773978, 0.00395584, 0.00213885, 0.00203657,
       0.00333452, 0.00329947, 0.00196052, 0.00337625, 0.00333595],
      dtype=float32)

In [11]:
#qpu = qml.device('qiskit.ibmq', wires=2, backend='ibmq_burlington',shots=100)
#qpu = qml.device('qiskit.ibmq', wires=2, backend='ibmq_essex',shots=100)
#qpu = qml.device('qiskit.ibmq', wires=2, backend='ibmq_london',shots=100)
#qpu = qml.device('qiskit.ibmq', wires=2, backend='ibmq_vigo',shots=100)
#qpu = qml.device('strawberryfields.fock', wires=1, cutoff_dim=10)
#qpu = qml.device('strawberryfields.gaussian', wires=1)
qpu = qml.device('qiskit.aer', wires=2)
#qpu=qml.device('default.qubit', wires=2)

In [12]:
def H_phase(x):
    for i in range(x.size): 
        qml.Hadamard(wires=i)
        qml.RZ(x[i],wires=i)
    

In [13]:
def entangle_rotate(x):
    for i in range(x.size): 
        i_next=(i+1)%(x.size)
        #prevent reverse cycle for two qbit system
        if i==1 and i_next==0:
            #print('state prep: preventing trivial reverse cycle for a 2 qbit circuit')
            break
            
        phi=(np.pi-x[i])*(np.pi-x[i_next])
        #print('entangling {} and {} with phi rotation {}'.format(i,i_next,phi))
        qml.CNOT(wires=[i,i_next])
        qml.RZ(phi,wires=i_next)
        qml.CNOT(wires=[i,i_next])



In [14]:
def ryrz(params):
    n_wire=params.size//2
    for i_wire in range(n_wire):
        i_theta=i_wire*2
        i_phi=i_theta+1
        qml.RY(params[i_theta],wires=i_wire)
        qml.RZ(params[i_phi],wires=i_wire) 

In [15]:
def cycle_ent(n_wires):
    for i in range(n_wires): 
        i_next=(i+1)%(n_wires)
        if i==1 and i_next==0:
            #print('classifier: preventing trivial reverse cycle for a 2 qbit circuit')
            break
        #print('entangling {} and {}'.format(i,i_next))
        qml.CNOT(wires=[i,i_next])

In [16]:
def make_layers(params,depth,n_wires):
    #print('params size {} and xcheck {}'.format(params.size,((depth+1)*n_wires*2)))
    assert params.size==((depth+1)*n_wires*2)
    ryrz(params[:2*n_wires])
    for i_layer in range(depth):
        i_p_first=(i_layer+1)*2*n_wires
        i_p_last=(i_layer+2)*2*n_wires #slice indexing, so one past
        cycle_ent(n_wires)
        ryrz(params[i_p_first:i_p_last])
    

In [17]:
@qml.qnode(qpu,interface='torch')
def circuit(params,inputs=None):
    #print(inputs)
    #qml.BasisState(np.asarray([0,0]),wires=[0,1])
    H_phase(inputs)
    entangle_rotate(inputs)
    make_layers(params,3,2)
    return [qml.expval(qml.PauliZ(0)),qml.expval(qml.PauliZ(1))]

In [18]:
from torch.nn.parameter import Parameter

In [19]:
import torch.nn as nn

In [20]:
class QCircuitNet(torch.nn.Module):
    def __init__(self):
        super(QCircuitNet, self).__init__()
        self.params=Parameter(torch.rand(16,dtype=torch.float32))
        self.fc1 = nn.Linear(2, 2)
        
    def forward(self,args):
        #call circuit on each example; build results tensor succesively
        #this seems very pedestrian but idk if there is a better way
        #print('args: {}'.format(args))
        exp_vals_list=[]
        for i_eval in range(args.size()[0]):
            exp_vals_list.append(circuit(self.params,inputs=args[i_eval]).float())
                                 
        exp_vals=torch.stack(exp_vals_list,dim=0)
        #print('exp_vals: {}'.format(exp_vals))
        return self.fc1(exp_vals)

In [21]:
model=QCircuitNet()

In [22]:
for p in model.named_parameters():
    print(p)

('params', Parameter containing:
tensor([0.8494, 0.7254, 0.5771, 0.4051, 0.1903, 0.5712, 0.2751, 0.5391, 0.2247,
        0.8397, 0.3724, 0.7454, 0.3517, 0.3427, 0.5570, 0.2527],
       requires_grad=True))
('fc1.weight', Parameter containing:
tensor([[ 0.6669, -0.2146],
        [-0.3466,  0.2362]], requires_grad=True))
('fc1.bias', Parameter containing:
tensor([-0.1708,  0.1919], requires_grad=True))


In [23]:
loss = nn.CrossEntropyLoss()

In [24]:
ti=iter(train_loader)

In [25]:
data, labels =ti.next()

In [26]:
data, labels

(tensor([[ 1.3294, -0.3476],
         [ 0.1064, -0.0518],
         [ 0.7542,  0.8224],
         [ 0.3513, -0.5336]]), tensor([1, 1, 0, 1]))

In [27]:
model.params

Parameter containing:
tensor([0.8494, 0.7254, 0.5771, 0.4051, 0.1903, 0.5712, 0.2751, 0.5391, 0.2247,
        0.8397, 0.3724, 0.7454, 0.3517, 0.3427, 0.5570, 0.2527],
       requires_grad=True)

In [29]:
data[0]

tensor([ 1.3294, -0.3476])

In [32]:
print(circuit(model.params,inputs=data[0]))
print(circuit(model.params,inputs=data[1]))
print(circuit(model.params,inputs=data[2]))
print(circuit(model.params,inputs=data[3]))

tensor([ 0.2227, -0.3457], dtype=torch.float64, grad_fn=<_TorchQNodeBackward>)
tensor([0.5625, 0.3477], dtype=torch.float64, grad_fn=<_TorchQNodeBackward>)
tensor([-0.5664, -0.0273], dtype=torch.float64, grad_fn=<_TorchQNodeBackward>)
tensor([0.4453, 0.3867], dtype=torch.float64, grad_fn=<_TorchQNodeBackward>)


In [31]:
resbatch=model(data)

args: tensor([[ 1.3294, -0.3476],
        [ 0.1064, -0.0518],
        [ 0.7542,  0.8224],
        [ 0.3513, -0.5336]])
exp_vals: tensor([[ 0.2168, -0.3359],
        [ 0.6094,  0.3965],
        [-0.4844, -0.0840],
        [ 0.4316,  0.3750]], grad_fn=<StackBackward>)
